## 先尋找職業所需的工作技能

## 先爬取外層頁

In [2]:
import requests
import re
from bs4 import BeautifulSoup
import jieba
from jieba.analyse import extract_tags
from collections import Counter
import json

In [3]:
#取得爬取網頁資訊所需的form data
payload = """find_key1:{}
strrec:
search_key_word:{}
search_type:job
us_menu:
search_item:1
search_from:index""".format("程式設計師","程式設計師")  #format內容為要查詢的職業

In [4]:
#使用一個function將form data轉換成dict的key-value pair才可放到requests的query中
def str2dict(data_str):
    data = {}
    for row in data_str.split('\n'):
        kv_list = row.split(":")
        data[kv_list[0]] = kv_list[1]
    return data

In [5]:
#測試一下看form data是否正確轉換成dict的key-value pair
data = str2dict(payload)
data

{'find_key1': '程式設計師',
 'search_from': 'index',
 'search_item': '1',
 'search_key_word': '程式設計師',
 'search_type': 'job',
 'strrec': '',
 'us_menu': ''}

In [6]:
def total_pages(html,formdata):
    formdata = str2dict(formdata)
    res = requests.post(html,data = formdata)
    res.encoding = 'utf8'
    soup = BeautifulSoup(res.text,'lxml')
    td_tag = soup.select_one('#top_title').find(lambda x: x.get('align') == 'right')
    pages = int(re.findall('\d+',td_tag.text)[2])
    return pages

In [7]:
all_pages = total_pages("https://www.yes123.com.tw/admin/job_refer_list.asp",payload)

In [8]:
# Pagination

res = requests.post("https://www.yes123.com.tw/admin/job_refer_list.asp",data=data)
res.encoding = 'utf8'
soup = BeautifulSoup(res.text,'lxml')
td_tag = soup.select_one("#top_title").find(lambda x: x.get('align') == 'right')
td_tag

<td align="right">
                            403 筆，
							第1 / 21頁
					</td>

In [9]:
total_records = int(re.findall('\d+',td_tag.text)[0])
total_records

403

In [10]:
#算出總頁數
total_page = int(total_records / 20) + 1
total_page

21

In [11]:
#算出最後一頁有幾筆
last_page = int(total_records % 20)
last_page

3

In [12]:
Links = []
for i in range(0,all_pages):
    data['strrec'] = i*20
    # data = str2dict(payload)
    # data

    res = requests.post("https://www.yes123.com.tw/admin/job_refer_list.asp",data=data)
    res.encoding = 'utf8'
    #     res.text

    soup = BeautifulSoup(res.text,'lxml')
        # 取得所有工作職缺
    titles =[job['title'] for job in soup.select('a.jobname')]


        # 取得所有工作職缺的連結
    urls = [job['href'] for job in soup.select('a.jobname')]
    uri = "https://www.yes123.com.tw/admin/"
    for links in urls:
        Links.append(uri+links)
#     job_list = []
#     for title,url in zip(titles, urls):
#         job_dict = {}
#         job_dict['title']  = title
#         job_dict['url'] = uri + url
#         job_list.append(job_dict)
#     Links.append(job_list)
# Links
#         Links.append(job_list)


In [13]:
len(Links)

403

In [14]:
cnt = Counter()
notChinese = re.compile('[0-9a-zA-Z#.]')
for Link in Links:
    try:
        all_links = Link
        url2 = all_links
        res2 = requests.get(url2)
        soup = BeautifulSoup(res2.text, 'lxml')
           

        for pcskill in soup.select('.comp_detail > ul > li'):
            
            if re.findall('電腦技能|工作技能',pcskill.text) and not re.findall('\d+\.',pcskill.text) :
                skill =pcskill.text.replace('、',',').replace('：',',').replace('/',',').replace('／',',').replace(',,',',').replace("\n",",").replace(" ",",").strip()
                
#                 print(skill.split(','))

                for skills in skill.split(','):
                    skills = "".join(notChinese.findall(skills))
#                     if re.findall('\w+',skills):
                    skills =skills.lower()
                    
                    if skills in cnt:
                        cnt[skills] += 1
                    else:
                        cnt[skills] = 1
#                             print (skills)
             
    except:
        pass

# print (cnt.most_common(100)[1:]) #因為第一名為沒有特別註記所需技能所以跳過
                    

In [15]:
import json
json.dump(cnt.most_common(100),open("Yes123_JobCount.json","w"),ensure_ascii=False)